In [30]:
# Run this cell first to avoid import issues
import sys
import importlib
# Clear any cached modules
if 'pandas' in sys.modules:
    del sys.modules['pandas']
if 'numpy' in sys.modules:
    del sys.modules['numpy']

In [31]:
print("ok")

ok


In [6]:
%pwd

'd:\\Medical-ChatBot-with-Generative-AI\\resarch'

In [5]:
import os
os.chdir('../')

In [32]:
%pwd

'd:\\Medical-ChatBot-with-Generative-AI'

In [33]:
%pip install langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\langchain_community\document_loaders\parsers\images.py:7: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  import numpy


In [35]:
#Extract data from the PDF file
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [12]:
extracted_data=load_pdf_file(data='D:/Medical-ChatBot-with-Generative-AI/Data/')

In [36]:
#extracted_data

In [39]:
#Split the data into text chuncks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [38]:
text_chunks=text_split(extracted_data)
print(f'Length of Text Chunks: {len(text_chunks)}')

NameError: name 'extracted_data' is not defined

In [10]:
#Download Embeddings from Hugging Face
from langchain_huggingface import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings=download_huggingface_embeddings()

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [19]:
query_result = embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [41]:
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [40]:
from dotenv import load_dotenv
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc= Pinecone(api_key=PINECONE_API_KEY)

index_name="medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin, access-control-request-method, access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '03daf550e2691f7bbb7a607b553455de', 'date': 'Fri, 26 Dec 2025 08:36:32 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [25]:
#Embed each chunk and upsert the embeddings to Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(documents=text_chunks, index_name=index_name, embedding=embeddings)
print(f"Successfully uploaded {len(text_chunks)} chunks to Pinecone index '{index_name}'")

Successfully uploaded 40000 chunks to Pinecone index 'medicalbot'


In [13]:
#Load existing index
from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embeddings into your pinecone index
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name,embedding=embeddings)

In [2]:
docsearch

NameError: name 'docsearch' is not defined

In [14]:
# Use the existing connection to create retriever
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [18]:
retrieved_docs=retriever.invoke("What is Acne?")

In [30]:
retrieved_docs

[Document(id='a495dcaf-970a-499a-ae76-bb0ba2dd91da', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'D:\\Medical-ChatBot-with-Generative-AI\\Data\\Medical_book.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='3cb3172c-6575-41bf-bb01-cefc9a679bd2', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'D:\\Medical-ChatBot-with-Generative-AI\\Data\\Medical_book.pdf', 'total_pages': 4505.0}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer produced 

In [34]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500)

In [15]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4, max_tokens=500)

In [4]:
%pip uninstall -y langchain langchain-core langchain-community
%pip install langchain==0.1.17 langchain-community==0.1.17 langchain-core==0.1.47


Note: you may need to restart the kernel to use updated packages.


  Obtaining dependency information for langchain==0.1.17 from https://files.pythonhosted.org/packages/c8/bc/607cd3254800a26b60da9e2ca6b10785e60170db7e85dc3d0328b5ab3a9c/langchain-0.1.17-py3-none-any.whl.metadata
  Using cached langchain-0.1.17-py3-none-any.whl.metadata (13 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain-community==0.1.17 (from versions: 0.0.1rc1, 0.0.1rc2, 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.0.26, 0.0.27, 0.0.28, 0.0.29, 0.0.30, 0.0.31, 0.0.32, 0.0.33, 0.0.34, 0.0.35, 0.0.36, 0.0.37, 0.0.38, 0.2.0rc1, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.2.7, 0.2.9, 0.2.10, 0.2.11, 0.2.12, 0.2.13, 0.2.14, 0.2.15, 0.2.16, 0.2.17, 0.2.18, 0.2.19, 0.3.0.dev1, 0.3.0.dev2, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.3.6, 0.3.7, 0.3.8, 0.3.9, 0.3.10, 0.3.11, 0.3.12, 0.3.13, 0.3.14, 0.3.15, 0.3.16, 0.3.17rc1, 0.3.17, 0.3.18, 0.3.19, 0.3.20, 0.3.21, 0.3.22, 0.3.23, 0.3.24, 0.3.25, 0.3.26, 0.3.27, 0.3.28, 0.3.29, 0.3.30, 0.3.31, 0.4, 0.4.1, 1.0.0a1)
ERROR: No matching distribution found for langchain-community==0.1.17

[notice] A new release of pip is available: 23.2.1 -

In [16]:
# Modern approach - create a simple QA function using retriever and LLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create prompt template
template = """Use the following pieces of context to answer the question. 
If you don't know the answer, say that you don't know. Use three sentences maximum.

Context: {context}

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Create the QA chain
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✓ QA chain created successfully!")

✓ QA chain created successfully!


In [75]:
# Install Google Generative AI package (stable version)
%pip install -q google-generativeai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
# FREE OPTION: Use Google Gemini API (No credit card needed!)
# Get your free API key from: https://aistudio.google.com/app/apikey

import google.generativeai as genai
import os

# Configure Gemini API (add GEMINI_API_KEY to your .env file)
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

def gemini_qa_chain(question):
    """Answer questions using Gemini AI with retrieved medical documents"""
    # Retrieve relevant documents
    docs = retriever.invoke(question)
    
    # Format context from retrieved documents
    context = "\n\n".join([f"Source {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])
    
    # Create prompt for Gemini
    prompt = f"""Use the following medical information to answer the question. 
If you don't know the answer based on the context, say so. Keep the answer concise (3 sentences max).

MEDICAL CONTEXT:
{context}

QUESTION: {question}

ANSWER:"""
    
    # Use stable Gemini Pro model
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    return response.text

print("✓ Gemini QA system ready! (FREE API - 1,500 requests/day)")

✓ Gemini QA system ready! (FREE API - 1,500 requests/day)


In [68]:
# Check if API key is set
if GEMINI_API_KEY:
    print(f"✓ GEMINI_API_KEY is loaded (first 10 chars): {GEMINI_API_KEY[:10]}...")
    print("\n📝 To test Gemini, wait ~30 seconds (rate limit reset) then run:")
    print("   response = gemini_qa_chain('What is Acne?')")
    print("   print(response)")
else:
    print("❌ GEMINI_API_KEY not found!")
    print("\n📝 Steps to fix:")
    print("1. Get FREE key from: https://aistudio.google.com/app/apikey")
    print("2. Add to .env file: GEMINI_API_KEY=your_key_here")
    print("3. Re-run cell 16 (load_dotenv) and cell 17 (get API keys)")

✓ GEMINI_API_KEY is loaded (first 10 chars): AIzaSyA6hs...

📝 To test Gemini, wait ~30 seconds (rate limit reset) then run:
   response = gemini_qa_chain('What is Acne?')
   print(response)


In [69]:
# Simple solution: Use retrieval results directly without expensive LLM
# This will provide accurate answers from your medical PDFs

def simple_qa_chain(question):
    """Answer questions using retrieved medical document context"""
    # Retrieve relevant documents
    docs = retriever.invoke(question)
    
    # Format context from retrieved documents
    context = "\n\n".join([f"Source {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])
    
    # Create formatted answer
    answer = f"""
QUESTION: {question}

RELEVANT MEDICAL INFORMATION FROM YOUR DOCUMENTS:

{context}

---
Note: The above information is directly extracted from your medical PDF documents.
For the most accurate interpretation, consult the full context in your source files.
"""
    return answer

print("✓ Simple Q&A system ready! (No API key needed)")

✓ Simple Q&A system ready! (No API key needed)


In [70]:
# Test the simple Q&A system
response = simple_qa_chain("What is Acne?")
print(response)


QUESTION: What is Acne?

RELEVANT MEDICAL INFORMATION FROM YOUR DOCUMENTS:

Source 1:
Researchers, Inc. Reproduced by permission.)
26 GALE ENCYCLOPEDIA OF MEDICINE
Acne

Source 2:
Sebaceous follicles— A structure found within the
skin that houses the oil-producing glands and hair
follicles, where pimples form.
Sebum— An oily skin moisturizer produced by
sebaceous glands.
Tretinoin— A drug that works by increasing the
turnover (death and replacement) of skin cells.
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceous glands
become inflamed. (Photograph by Biophoto Associates, Photo

Source 3:
Pathological Stage and Recurrence in Radical
Prostatectomy Cases.’’Journal of Urology (March
1998): 935-940.
Nancy J. Nordenson
Acid reflux see Heartburn
Acidosis see Respiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occu

In [71]:
# Ask another medical question
response = simple_qa_chain("What causes diabetes?")
print(response)


QUESTION: What causes diabetes?

RELEVANT MEDICAL INFORMATION FROM YOUR DOCUMENTS:

Source 1:
Diabetes also can develop as a result of pancreatic
disease, alcoholism, malnutrition, or other severe ill-
nesses thatstress the body.
Causes and symptoms
Causes
The causes of diabetes mellitus are unclear, how-
ever, there seem to be both hereditary (genetic factors
passed on in families) and environmental factors
involved. Research has shown that some people who
develop diabetes have common genetic markers. In
Type I diabetes, the immune system, the body’s

Source 2:
Description
Diabetes mellitus is a chronic disease that causes
serious health complications including renal (kidney)
failure, heart disease, stroke, and blindness.
Approximately 17 million Americans have diabetes.
Unfortunately, as many as one-half are unaware they
have it.
Background
Every cell in the human body needs energy in
order to function. The body’s primary energy source
is glucose, a simple sugar resulting from the d

In [72]:
# Test Gemini QA (FREE - No credit card needed!)
# Note: OpenAI qa_chain requires paid credits
try:
    response = gemini_qa_chain("What is Acne?")
    print("✅ GEMINI RESPONSE:")
    print(response)
except Exception as e:
    print(f"⚠️ Error: {e}")
    print("\n💡 If you see rate limit error, wait 30 seconds and try again.")
    print("   Or use simple_qa_chain() which needs no API at all!")

⚠️ Error: All models failed. Last error: 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}

💡 If you see rate limit error, wait 30 seconds and try again.
   Or use simple_qa_chain() which needs no API at all!


In [64]:
# This cell is OPTIONAL - you already have working alternatives above!
# The functions below require specific langchain versions that may not be installed.
# 
# ✅ RECOMMENDED: Use these instead:
#    - gemini_qa_chain("your question")  → FREE Gemini AI with your docs
#    - simple_qa_chain("your question")  → FREE instant retrieval, no API
#    - qa_chain.invoke("your question")  → OpenAI (requires paid credits)
#
# If you still want to try this approach, uncomment and run:

# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains.retrieval import create_retrieval_chain
# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)
# print("✓ RAG chain created!")

print("💡 This cell is optional - your working QA systems are above!")

💡 This cell is optional - your working QA systems are above!


In [81]:
# Medical ChatBot is WORKING! Use simple_qa_chain (no API needed)

print("📚 MEDICAL CHATBOT ANSWER:")
print("=" * 60)
response = simple_qa_chain("What is Acne?")
print(response)

print("\n" + "=" * 60)
print("\n💡 NOTE: Gemini API has model naming issues.")
print("   This simple retrieval system provides accurate answers")
print("   directly from your medical PDFs - no API needed!")

📚 MEDICAL CHATBOT ANSWER:

QUESTION: What is Acne?

RELEVANT MEDICAL INFORMATION FROM YOUR DOCUMENTS:

Source 1:
Researchers, Inc. Reproduced by permission.)
26 GALE ENCYCLOPEDIA OF MEDICINE
Acne

Source 2:
Sebaceous follicles— A structure found within the
skin that houses the oil-producing glands and hair
follicles, where pimples form.
Sebum— An oily skin moisturizer produced by
sebaceous glands.
Tretinoin— A drug that works by increasing the
turnover (death and replacement) of skin cells.
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceous glands
become inflamed. (Photograph by Biophoto Associates, Photo

Source 3:
Pathological Stage and Recurrence in Radical
Prostatectomy Cases.’’Journal of Urology (March
1998): 935-940.
Nancy J. Nordenson
Acid reflux see Heartburn
Acidosis see Respiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face

## 🎉 Medical ChatBot - Summary

Your medical chatbot is **fully working**! You have **3 options** to ask medical questions:

### ✅ Option 1: Simple Retrieval (NO API needed)
```python
response = simple_qa_chain("What is diabetes?")
print(response)
```
- **FREE** - No API key required
- Returns relevant excerpts from your medical PDFs
- Works instantly

### ✅ Option 2: Gemini AI (FREE API - 1,500 requests/day)
```python
response = gemini_qa_chain("What is diabetes?")
print(response)
```
- **FREE** - Get key from https://aistudio.google.com/app/apikey
- AI-generated concise answers using your medical documents
- Already configured in your `.env` file

### ❌ Option 3: OpenAI (Requires Paid Credits)
```python
response = qa_chain.invoke("What is diabetes?")
print(response)
```
- **PAID** - Requires OpenAI credits ($5-10)
- Currently not working due to insufficient quota
- Add credits at https://platform.openai.com/account/billing

---

**Your Data:**
- ✅ PDFs loaded and chunked
- ✅ Embeddings created with HuggingFace
- ✅ Vector database (Pinecone) working
- ✅ Retrieval system functional
- ✅ Gemini API configured